In [22]:
import pandas as pd
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import RobustScaler

In [2]:
train = pd.read_csv("../input/processed/processed_train.csv")
test = pd.read_csv("../input/processed/processed_test.csv")

In [3]:
train

,stock_id,time_id,target,row_id,wap1_sum,wap1_mean,wap1_std,wap2_sum,wap2_mean,wap2_std,...,trade_size_sum_450,trade_order_count_mean_450,trade_log_return_realized_volatility_300,trade_seconds_in_bucket_count_unique_300,trade_size_sum_300,trade_order_count_mean_300,trade_log_return_realized_volatility_150,trade_seconds_in_bucket_count_unique_150,trade_size_sum_150,trade_order_count_mean_150
0,0,5,0.004136,0-5,303.12506,1.003725,0.000693,303.105530,1.003661,0.000781,...,1042.0,2.642857,0.001308,21.0,1587.0,2.571429,0.001701,30.0,2069.0,2.433333
1,0,11,0.001445,0-11,200.04778,1.000239,0.000262,200.041170,1.000206,0.000272,...,828.0,2.200000,0.000587,16.0,900.0,2.250000,0.000813,24.0,1173.0,2.041667
2,0,16,0.002168,0-16,187.91385,0.999542,0.000864,187.939820,0.999680,0.000862,...,1085.0,3.666667,0.001137,12.0,1189.0,3.166667,0.001621,20.0,2010.0,2.950000
3,0,31,0.002195,0-31,119.85978,0.998831,0.000757,119.835945,0.998633,0.000656,...,514.0,3.666667,0.001089,9.0,1556.0,5.111111,0.001401,11.0,1631.0,4.545455
4,0,62,0.001747,0-62,175.93286,0.999619,0.000258,175.934250,0.999626,0.000317,...,43.0,3.500000,0.000453,11.0,1219.0,4.909091,0.000550,16.0,1570.0,4.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
428927,126,32751,0.003461,126-32751,309.87045,0.999582,0.000486,309.871370,0.999585,0.000613,...,491.0,1.833333,0.001451,18.0,796.0,2.055556,0.001924,27.0,1426.0,2.259259
428928,126,32753,0.003113,126-32753,223.55214,1.002476,0.001264,223.580320,1.002602,0.001303,...,326.0,1.769231,0.001791,20.0,1107.0,3.550000,0.002101,31.0,1550.0,3.161290
428929,126,32758,0.004070,126-32758,256.27704,1.001082,0.000466,256.255070,1.000996,0.000599,...,348.0,2.166667,0.001580,24.0,2750.0,2.541667,0.001913,31.0,3493.0,2.838710
428930,126,32763,0.003357,126-32763,399.72174,1.001809,0.000456,399.714320,1.001790,0.000507,...,2300.0,2.727273,0.001520,43.0,5150.0,2.813953,0.001714,62.0,7261.0,2.822581


In [4]:
df = train.drop(['time_id', 'target', 'row_id'], axis=1)
df0 = df[df['stock_id']==0]
df0

,stock_id,wap1_sum,wap1_mean,wap1_std,wap2_sum,wap2_mean,wap2_std,log_return1_sum,log_return1_realized_volatility,log_return1_mean,...,trade_size_sum_450,trade_order_count_mean_450,trade_log_return_realized_volatility_300,trade_seconds_in_bucket_count_unique_300,trade_size_sum_300,trade_order_count_mean_300,trade_log_return_realized_volatility_150,trade_seconds_in_bucket_count_unique_150,trade_size_sum_150,trade_order_count_mean_150
0,0,303.12506,1.003725,0.000693,303.105530,1.003661,0.000781,0.002292,0.004499,7.613697e-06,...,1042.0,2.642857,0.001308,21.0,1587.0,2.571429,0.001701,30.0,2069.0,2.433333
1,0,200.04778,1.000239,0.000262,200.041170,1.000206,0.000272,0.000360,0.001204,1.810376e-06,...,828.0,2.200000,0.000587,16.0,900.0,2.250000,0.000813,24.0,1173.0,2.041667
2,0,187.91385,0.999542,0.000864,187.939820,0.999680,0.000862,-0.002074,0.002369,-1.109168e-05,...,1085.0,3.666667,0.001137,12.0,1189.0,3.166667,0.001621,20.0,2010.0,2.950000
3,0,119.85978,0.998831,0.000757,119.835945,0.998633,0.000656,-0.002828,0.002574,-2.376725e-05,...,514.0,3.666667,0.001089,9.0,1556.0,5.111111,0.001401,11.0,1631.0,4.545455
4,0,175.93286,0.999619,0.000258,175.934250,0.999626,0.000317,-0.000002,0.001894,-1.021740e-08,...,43.0,3.500000,0.000453,11.0,1219.0,4.909091,0.000550,16.0,1570.0,4.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3825,0,296.38748,0.997938,0.000747,296.365480,0.997864,0.000769,-0.002136,0.002579,-7.215169e-06,...,1159.0,2.947368,0.001162,35.0,2365.0,3.257143,0.001409,42.0,2957.0,3.238095
3826,0,206.06390,1.000310,0.000551,206.100400,1.000487,0.000599,0.000403,0.002206,1.966603e-06,...,1158.0,4.600000,0.001066,12.0,2161.0,4.250000,0.001284,19.0,2494.0,3.421053
3827,0,187.91570,0.999552,0.000743,187.897700,0.999456,0.000736,0.001663,0.002913,8.895233e-06,...,416.0,2.000000,0.001242,22.0,2294.0,3.727273,0.001375,27.0,2736.0,3.444444
3828,0,307.72370,1.002357,0.000356,307.732640,1.002387,0.000424,0.000520,0.003046,1.698934e-06,...,1415.0,2.666667,0.001404,25.0,1627.0,1.920000,0.001650,36.0,2296.0,2.055556


In [9]:
df0 = df0.fillna(0)

In [27]:
transformer = RobustScaler().fit(df0)
X = transformer.transform(df0)
X

array([[ 0.        ,  0.80809446,  1.72704808, ...,  0.57142857,
         0.06583984, -0.53253012],
       [ 0.        , -0.31693423,  0.09374012, ...,  0.14285714,
        -0.48292758, -0.85341365],
       [ 0.        , -0.44936903, -0.23284652, ..., -0.14285714,
         0.02970449, -0.10923695],
       ...,
       [ 0.        , -0.44934884, -0.2282696 , ...,  0.35714286,
         0.47435309,  0.29585007],
       [ 0.        ,  0.85828595,  1.0860924 , ...,  1.        ,
         0.20486909, -0.84203481],
       [ 0.        , -0.01403463, -0.42908708, ...,  0.07142857,
         0.8479559 , -0.28199756]])

In [47]:
clustering = DBSCAN(eps=10, min_samples=2).fit(X)

In [48]:
np.histogram(clustering.labels_)

(array([3806,    2,    2,    2,    2,    2,    2,    2,    5,    5]),
 array([-1. ,  0.1,  1.2,  2.3,  3.4,  4.5,  5.6,  6.7,  7.8,  8.9, 10. ]))

In [26]:
def l2(x1, x2):
    return np.sqrt(np.sum((x1-x2)**2))

In [30]:
l2(X[2], X[1])

9.079515503677387